# Import Necessary Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import MaxPool2D
from keras.layers import Flatten, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet



# Image Augmentation

In [4]:
train_datagenerator = ImageDataGenerator(rescale= 1 / 255.0,
                                 horizontal_flip= True, 
                                 vertical_flip= True, 
                              
                                 )
                        
valid_datagenerator = ImageDataGenerator(rescale= 1 / 255.0,
                                 horizontal_flip= True, 
                                 vertical_flip= True, 
                                 
                                 )
                            

In [5]:
input_shape = (224,224)
train_dataset = train_datagenerator.flow_from_directory('dataset/train',
                                                       shuffle= True, 
                                                       class_mode= 'categorical',
                                                       batch_size= 32,
                                                       target_size=  input_shape)
validation_dataset = train_datagenerator.flow_from_directory('dataset/validate',
                                                       shuffle= True, 
                                                       class_mode= 'categorical',
                                                       batch_size= 32,
                                                       target_size=  input_shape)

Found 5574 images belonging to 4 classes.
Found 1097 images belonging to 4 classes.


## Transfer learning

In [6]:
model_base = MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [7]:

model = Sequential()
model.add(model_base)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(4,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 4100      
Total params: 3,232,964
Trainable params: 3,211,076
Non-trainable params: 21,888
_________________________________________________________________


# compiling the model

In [8]:
LOSS  = 'categorical_crossentropy'
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = LOSS, 
             metrics=['accuracy'])

# Saving the model with best validation accuracy

In [10]:
save_path = "final_model4.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath= save_path,
                                           monitor= 'val_accuracy',
                                           mode='max',
                                           save_best_only=True    )

# Fitting the model

In [11]:
EPOCHS = 12
VALIDATION_DATA = validation_dataset
VERBOSE = 2
BATCH_SIZE = 32
STEP_PER_EPOCH = 32
# training model
history = model.fit(train_dataset, verbose= VERBOSE,
                    epochs=EPOCHS, steps_per_epoch=STEP_PER_EPOCH, validation_data= VALIDATION_DATA,
                    callbacks = [checkpoint])

Epoch 1/12
 - 1041s - loss: 0.2898 - accuracy: 0.9038 - val_loss: 2.2200 - val_accuracy: 0.6007


/home/bhabuk/.local/lib/python3.7/site-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Epoch 2/12
 - 1074s - loss: 0.1553 - accuracy: 0.9580 - val_loss: 0.6931 - val_accuracy: 0.9006
Epoch 3/12
 - 964s - loss: 0.0660 - accuracy: 0.9766 - val_loss: 1.2016 - val_accuracy: 0.8459
Epoch 4/12
 - 1159s - loss: 0.0419 - accuracy: 0.9873 - val_loss: 0.9044 - val_accuracy: 0.9517
Epoch 5/12
 - 949s - loss: 0.0607 - accuracy: 0.9844 - val_loss: 7.4403e-04 - val_accuracy: 0.9407
Epoch 6/12


KeyboardInterrupt: 